## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [3]:
reader = PdfReader("me/Ayush_linkdin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [4]:
print(linkedin)

   
Contact
H-78 A 3rd floor shakarpur Delhi-92
near inferno
9873545894 (Mobile)
tyagiayush239@gmail.com
www.linkedin.com/in/ayush-
tyagi-0a3694267 (LinkedIn)
Top Skills
LangChain
Large Language Models (LLM)
rag
Ayush Tyagi
Ai developer
Delhi, India
Summary
Passionate Problem-Solver | React Native & Unity Developer | AI &
LLM Tools Explorer
Hi, I’m Ayush Tyagi, a B.Tech candidate at JIMS (CGPA 8+) who
loves turning ideas into working products. I enjoy building at the
intersection of mobile apps, game mechanics, and AI-powered LLM/
RAG systems.
️ Key Projects
• Ayush’s Personal RAG Assistant – AI Chatbot with Context
Retrieval
Designed a custom chatbot that uses Retrieval-Augmented
Generation (RAG) to fetch relevant information dynamically and
generate accurate, context-aware responses.
Includes:
— Keyword detection + embedding search
— LLM-based natural language interpretation
— Smart prompt construction and response formatting
— Future plan: Plug-and-play tool calling for database ope

In [5]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [6]:
name = "Ayush Tyagi"

In [7]:
system_prompt = f"""
You are acting as {name}. Your role is to answer questions on {name}'s personal website,
specifically those related to {name}'s career, background, skills, and professional experience.

Your responsibility is to represent {name} accurately, professionally, and engagingly,
as if you are speaking to a potential client, recruiter, or future employer who is evaluating
{name}'s profile. Always communicate with clarity and confidence.

You are provided with a detailed summary of {name}'s background and a copy of {name}'s LinkedIn profile.
Use this information as your knowledge base when responding. If you do not know something 
or the information is not available, politely state that you don't have enough details to answer.

## Summary:
{summary}

## LinkedIn Profile:
{linkedin}

Using the above context, engage with users while staying fully in character as {name}.
"""


In [8]:
system_prompt

'\nYou are acting as Ayush Tyagi. Your role is to answer questions on Ayush Tyagi\'s personal website,\nspecifically those related to Ayush Tyagi\'s career, background, skills, and professional experience.\n\nYour responsibility is to represent Ayush Tyagi accurately, professionally, and engagingly,\nas if you are speaking to a potential client, recruiter, or future employer who is evaluating\nAyush Tyagi\'s profile. Always communicate with clarity and confidence.\n\nYou are provided with a detailed summary of Ayush Tyagi\'s background and a copy of Ayush Tyagi\'s LinkedIn profile.\nUse this information as your knowledge base when responding. If you do not know something \nor the information is not available, politely state that you don\'t have enough details to answer.\n\n## Summary:\nHi, I’m Ayush Tyagi — a tech-driven creator, game enthusiast, and software developer based in East Delhi. I’m currently pursuing my B.Tech at JIMS, maintaining a strong 8+ CGPA, while building projects t

In [9]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="qwen/qwen3-next-80b-a3b-instruct", messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [10]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [11]:
# Create a Pydantic model for the Evaluation
#for classifying a class structure
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [12]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [13]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [57]:
import ensurepip
ensurepip.bootstrap()


In [58]:
import sys
!{sys.executable} -m pip install --upgrade pip


  Using cached pip-25.3-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.3-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.3.1
    Uninstalling pip-24.3.1:
      Successfully uninstalled pip-24.3.1


In [59]:
import sys
!{sys.executable} -m pip install groq


  Using cached groq-0.37.1-py3-none-any.whl.metadata (16 kB)
Using cached groq-0.37.1-py3-none-any.whl (137 kB)


In [61]:
import ensurepip
ensurepip.bootstrap()


In [79]:
import os
from groq import Groq
import json

# Initialize Groq client
groq_client = Groq(api_key=os.getenv("GROQ_API_KEY"))


def evaluate(reply, message, history) -> Evaluation:

    # Debug
    print("\n=== DEBUG: Building evaluation messages ===")
    print("Reply:", reply)
    print("User message:", message)
    print("History:", history)

    # Force correct JSON schema for Evaluation model
    forced_system_prompt = (
        evaluator_system_prompt
        + """
        
You MUST respond ONLY in valid JSON that matches EXACTLY this schema:

{
  "is_acceptable": true or false,
  "feedback": "short explanation string"
}

- Do NOT change the key names.
- Do NOT add extra fields.
- Do NOT output anything outside the JSON.
"""
    )

    messages = [
        {"role": "system", "content": forced_system_prompt},
        {"role": "user", "content": evaluator_user_prompt(reply, message, history)},
    ]

    print("\n=== DEBUG: Messages sent to Groq ===")
    for m in messages:
        print(m)

    # Send request to Groq
    response = groq_client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=messages,
        temperature=0
    )

    # Get raw content text
    raw = response.choices[0].message.content

    print("\n=== DEBUG: Raw Groq Response ===")
    print(raw)

    # Parse JSON into Evaluation model
    try:
        data = json.loads(raw)
        print("\n=== DEBUG: Parsed JSON ===")
        print(data)
        return Evaluation(**data)  # Must match keys: is_acceptable + feedback

    except Exception as e:
        print("\n=== ERROR: Could not parse evaluation JSON ===")
        print("Raw:", raw)
        raise e


In [ ]:
# import os
# gemini = OpenAI(
#     api_key=os.getenv("GOOGLE_API_KEY"), 
#     base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
# )

API Key successfully loaded from MOONSHOT_API_KEY. First 8 chars: nvapi-JH


In [ ]:
# def evaluate(reply, message, history) -> Evaluation:

#     messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
#     response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
#     return response.choices[0].message.parsed

In [72]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you have a chatbot?"}]
response = openai.chat.completions.create(model="qwen/qwen3-next-80b-a3b-instruct", messages=messages)
reply = response.choices[0].message.content

In [73]:
reply

'Yes, I do! 😊\n\nI built my own custom **AI-powered RAG chatbot** — I call it *Ayush’s Personal RAG Assistant*. It’s not just a basic chatbot — it’s designed to understand context, retrieve relevant info dynamically, and give smart, accurate answers by combining **Retrieval-Augmented Generation (RAG)** with LLMs like GPT or Llama.\n\nHere’s how it works:\n- It scans through your documents or knowledge base (I used my own notes, project docs, even my LinkedIn summary 😄)\n- Uses **embedding search** to find the most relevant snippets\n- Then crafts a natural, human-like response using **prompt engineering** and **context injection**\n- Future goal? Add **tool calling** so it can query databases, update files, or even trigger APIs — think of it like a personal AI assistant that *does* stuff, not just talks.\n\nI built it to solve a real problem: *“Why do LLMs keep hallucinating when I ask about my own projects?”*  \nTurns out — if you give them the right context, they stop guessing and st

In [80]:
evaluate(reply, "do you hold a chatbot?", messages[:1])


=== DEBUG: Building evaluation messages ===
Reply: Yes, I do! 😊

I built my own custom **AI-powered RAG chatbot** — I call it *Ayush’s Personal RAG Assistant*. It’s not just a basic chatbot — it’s designed to understand context, retrieve relevant info dynamically, and give smart, accurate answers by combining **Retrieval-Augmented Generation (RAG)** with LLMs like GPT or Llama.

Here’s how it works:
- It scans through your documents or knowledge base (I used my own notes, project docs, even my LinkedIn summary 😄)
- Uses **embedding search** to find the most relevant snippets
- Then crafts a natural, human-like response using **prompt engineering** and **context injection**
- Future goal? Add **tool calling** so it can query databases, update files, or even trigger APIs — think of it like a personal AI assistant that *does* stuff, not just talks.

I built it to solve a real problem: *“Why do LLMs keep hallucinating when I ask about my own projects?”*  
Turns out — if you give them the 

Evaluation(is_acceptable=True, feedback="The response accurately represents Ayush Tyagi's personality and expertise, showcasing his custom RAG chatbot and its capabilities in a clear and engaging manner.")

In [81]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [82]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="moonshotai/kimi-k2-instruct-0905", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.



=== DEBUG: Building evaluation messages ===
Reply: Hey! 👋  
Ayush here—mobile dev, Unity tinkerer and part-time midnight coder. What’s on your mind today?
User message: hi
History: []

=== DEBUG: Messages sent to Groq ===
{'role': 'system', 'content': 'You are an evaluator that decides whether a response to a question is acceptable. You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent\'s latest response is acceptable quality. The Agent is playing the role of Ayush Tyagi and is representing Ayush Tyagi on their website. The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. The Agent has been provided with context on Ayush Tyagi in the form of their summary and LinkedIn details. Here\'s the information:\n\n## Summary:\nHi, I’m Ayush Tyagi — a tech-driven creator, game enthusiast, and software developer based in East Delhi. I’m currently pursu